<a href="https://colab.research.google.com/github/jinyoungkim0214/pytorch-highlevel-apis/blob/main/PyTorch_high_level_apis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (C) 2020 Software Platform Lab, Seoul National University

Licensed under the Apache License, Version 2.0 (the "License");

you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software

distributed under the License is distributed on an "AS IS" BASIS,


WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.


See the License for the specific language governing permissions and


limitations under the License.

### Preparing MNIST Dataset

Pytorch is providing a MNIST Dataset class, so we can simply import it from `torchvision.datasets` without handcrafting it. Before instantiating Dataset objects, let's first define transforms for MNIST dataset. `torchvision.transforms` module contains multiple predefined transforms. Here, we apply `ToTensor()` that converts the data into the PyTorch Tensor type and `Normalize(mean, std)` that normalizes the sample data to have given mean and standard deviation.

In [ ]:
import torchvision.transforms as transforms
#컴퓨터비전을 위한 라이브러리
#웬만한건 다 있음

# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

We use `torchvision.datasets.MNIST` API to instantiate MNIST Dataset objects. Having `download=True` flag, MNIST dataset will be automatically downloaded at `download_root` unless it already exists.

In [ ]:
from torchvision.datasets import MNIST

# download path
download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.10MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.00MB/s]


Finally, we instantiate DataLoader that can shuffle and batch the MNIST Dataset.

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True)

imgs, labels = next(iter(train_loader))
print(imgs.shape, labels.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])


## Custom Neural Network Models

PyTorch `torch.nn.Module` allows you to easily make your own custom neural network model. All you need to do is to 1) make a class that inherits `torch.nn.Module` class and 2) define `forward` method. Let's build a simple CNN model using `torch.nn` APIs.

* `torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')`
* `torch.nn.Linear(in_features, out_features, bias=True)`
* `torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)`
* `torch.nn.functional.relu(input, inplace=False)`
* `torch.nn.functional.softmax(input, dim=None)`

You can refer to the following links for more detailed descriptions of `torch.nn` APIs.
* https://pytorch.org/docs/stable/nn.html
* https://pytorch.org/docs/stable/nn.functional.html

In [ ]:
import torch
import torch.nn as nn #neuralnetwork_신경망미리구현
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1= nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2) #커널사이즈가 커지면 파라미터가 증가한다, 3*3 쓰면 파라미터가 줄기 때문에 레이어를 깊게 쌓기 가능
        self.pool1 = nn.MaxPool2d(2) #정보손실(downsampling):채널을 늘려줘야 함, 각 채널에 저장

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1) #2dconv바로가져올수있음 #3*3이니까padding=1일때output사이즈유지
        self.pool3 = nn.MaxPool2d(2)

        #self.conv4 = nn.Conv2d(32,64, 5, 1)
        #self.pool4 = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(288, 128)
        self.dropout = nn.Dropout(0.3) #overfitting방지
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,10)

        self.bn1 = nn.BatchNorm2d(6)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)

    def forward(self, x):
        #input size ; 1*28*28
        # First convolution layer
        x = self.conv1(x)
        #size:6*28*28
        #batch normalization
        x = self.bn1(x)
        x = F.relu(x) #max{x,0}이미구현
        x = self.pool1(x)
        #6*14*14


        # Second convolution layer
        x = self.conv2(x)
        #16*14*14
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
        #16*7*7

        # Third convolution layer
        x = self.conv3(x)
        #32*7*7
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)
        #32*3*3 (3.5->3)

        # (N, 256) flatten과 동일한 효과
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x #loss를crossentropy로쓰니까

      #  return F.soft(x, dim=1) #아웃풋은확률_다더해서1

Instantiate the custom neural network model.

In [ ]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=288, out_features=128, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


Define the loss function (`criterion`) and the optimization method (`optimizer`). In this example, cross entropy loss is used as the criterion and SGD is used as the optimizer. By having `net.parameters()` as the input for the optimizer, we are trying to apply SGD to all the trainable parameters that consist of our custom neural network model `net`.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 학습을 하려는 파라미터 :net.parameters()
# 특정 파라미터만 학습하고자하면 변경 필요
# 학습률 : 0.01(alpha)

Finally, connecting altoghether, we can train the model.

In [ ]:


num_epochs = 10

for epoch in range(num_epochs):
    train_loss = 0.0
    # Iteration over the train dataset
    for i, data in enumerate(train_loader):
        x, label = data
        # 1. Initalize gradient values
        optimizer.zero_grad() # 해야하는 이유 :

        # 2. Forward propagation
        model_output = net(x)

        # 3. Calculate loss using the criterion
        loss = criterion(model_output, label)  # 계산 시 위에서 설정한 크로스엔트로피 사용

        # 4. Back propagation
        loss.backward()

        # 5. Weight update
        optimizer.step()   # weight 업데이트 일어남

        train_loss += loss.item()  # 각 loss를 저장


    # Print train loss and test accuracy at the end of every epoch
    with torch.no_grad(): # do not forget this
        corr_num = 0
        total_num = 0
        # Iteration overt the test dataset to evaluate the test accuracy
        for _, test in enumerate(test_loader):
            test_x, test_label = test
            test_output = net(test_x)
            pred_label = test_output.argmax(dim=1)
            corr = test_label[test_label == pred_label].size(0)
            corr_num += corr
            total_num += test_label.size(0)
    print("[Epoch: %d] train loss: %.4f, test acc: %.2f" \
        % (epoch + 1, train_loss / len(train_loader), corr_num / total_num * 100))
    train_loss = 0.0


[Epoch: 1] train loss: 0.8558, test acc: 94.26
[Epoch: 2] train loss: 0.1558, test acc: 97.03
[Epoch: 3] train loss: 0.1022, test acc: 97.63
[Epoch: 4] train loss: 0.0816, test acc: 97.68
[Epoch: 5] train loss: 0.0700, test acc: 98.09
[Epoch: 6] train loss: 0.0605, test acc: 98.16
[Epoch: 7] train loss: 0.0550, test acc: 98.43
[Epoch: 8] train loss: 0.0482, test acc: 98.23
[Epoch: 9] train loss: 0.0445, test acc: 98.60
[Epoch: 10] train loss: 0.0410, test acc: 98.43
